[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/xbrl-financial-data-python.ipynb)

# Parse XBRL Financial Data from SEC EDGAR Filings with Python -- Free, No API Key

Use **edgartools** to parse XBRL financial data from SEC EDGAR filings -- completely free, no API key or paid subscription required. Every 10-K and 10-Q contains XBRL-tagged financial data including income statements, balance sheets, cash flows, and dozens of disclosure schedules.

**What you'll learn:**
- Parse XBRL data from any SEC filing
- Browse all available financial statements and disclosures
- Search for specific XBRL data (debt, leases, segments, etc.)
- Extract income statements, balance sheets, and cash flows
- Convert XBRL statements to pandas DataFrames
- Explore disclosure schedules beyond the main financial statements

## Install edgartools

In [ ]:
!pip install -U edgartools

## Setup

The SEC requires all automated tools to identify themselves. Replace the email below with your own -- any valid email works.

In [ ]:
from edgar import *

# The SEC requires you to identify yourself (any email works)
set_identity("your.name@example.com")

## Parse XBRL from a 10-K in 3 Lines

Every 10-K filing contains structured XBRL data. edgartools parses it and gives you access to all financial statements and disclosures:

In [ ]:
filing = Company("MSFT").get_filings(form="10-K")[0]
xbrl = filing.xbrl()

xbrl

## Browse All Available Statements

XBRL filings contain far more than the three main financial statements. Use `list_available()` to see everything:

In [ ]:
available = xbrl.statements.list_available()

# Count by category
print(f"Total statements: {len(available)}")
for cat, count in available.groupby("category").size().items():
    print(f"  {cat:15s} {count:>3}")

# Show the main financial statements
print("\nMain financial statements:")
main = available[available["category"] == "statement"]
for _, row in main.iterrows():
    print(f"  {row['name']:55s} {row['element_count']:>3} items")

## Income Statement

Search for statements by keyword. The `search()` method returns matching Statement objects with automatic formatting:

In [ ]:
income_statements = xbrl.statements.search("income")
print(f"Found {len(income_statements)} income-related statements\n")

# The first result is the main income statement
income_statements[0]

## Balance Sheet and Cash Flow

Search for any statement type -- balance sheets, cash flows, equity statements:

In [ ]:
balance_sheets = xbrl.statements.search("balance")
balance_sheets[0]

In [ ]:
cashflows = xbrl.statements.search("cash")
cashflows[0]

## Convert XBRL Statements to DataFrames

Every Statement object has a `.to_dataframe()` method for pandas analysis. Date columns contain the financial values:

In [ ]:
income = xbrl.statements.search("income")[0]
df = income.to_dataframe()

# Period columns contain financial values
date_cols = [c for c in df.columns if c.startswith("20")]
print(f"Shape: {df.shape}")
print(f"Periods: {date_cols}\n")

# Key line items (skip header rows)
df[~df["abstract"]][["label"] + date_cols].head(10)

## Explore Disclosure Schedules

Beyond the main statements, XBRL data includes detailed disclosures on debt, leases, segments, and more. Search by topic to find them:

In [ ]:
# Search for different disclosure topics
for topic in ["debt", "lease", "segment", "revenue", "tax"]:
    results = xbrl.statements.search(topic)
    print(f"  '{topic}': {len(results)} disclosures found")

# Look at a specific disclosure -- segment data
segments = xbrl.statements.search("segment")
print(f"\nSegment disclosures: {len(segments)}")

# Get the largest segment disclosure (most data)
largest = max(segments, key=lambda s: len(s.to_dataframe()))
largest

## Shortcut: The Financials Object

For quick access to the main financial statements, use `get_financials()` which wraps XBRL parsing into a clean interface:

In [ ]:
financials = Company("MSFT").get_financials()

# Direct access to statements
print(f"Revenue:     ${financials.get_revenue():>18,.0f}")
print(f"Net Income:  ${financials.get_net_income():>18,.0f}")
print(f"Total Assets:${financials.get_total_assets():>18,.0f}")

# Also available:
# financials.income_statement()
# financials.balance_sheet()
# financials.cashflow_statement()

## Why EdgarTools?

EdgarTools is free and open-source. Compare parsing XBRL data:

**With edgartools (free, no API key):**
```python
xbrl = Company("MSFT").get_filings(form="10-K")[0].xbrl()
xbrl.statements.search("income")[0]     # Formatted statement
xbrl.statements.search("debt")          # Debt disclosures
statement.to_dataframe()                 # As DataFrame
```

**Typical approach (manual XBRL parsing):**
```python
from lxml import etree
# ... download XBRL instance + taxonomy + linkbases,
# ... parse XML namespaces, resolve element references,
# ... map XBRL concepts to line items, handle dimensions manually
```

With edgartools, XBRL parsing is automatic -- all statements searchable and formatted, no XML knowledge required.

## Quick Reference

```python
from edgar import *
set_identity("your.name@example.com")

# ── Parse XBRL from a filing ──
filing = Company("MSFT").get_filings(form="10-K")[0]
xbrl = filing.xbrl()

# ── Browse all statements ──
xbrl.statements.list_available()           # DataFrame of all statements
len(xbrl.statements)                       # Total count

# ── Search by keyword ──
xbrl.statements.search("income")           # Income statements
xbrl.statements.search("balance")          # Balance sheets
xbrl.statements.search("cash")             # Cash flow statements
xbrl.statements.search("debt")             # Debt disclosures
xbrl.statements.search("segment")          # Segment data

# ── Work with a statement ──
stmt = xbrl.statements.search("income")[0]
stmt                                        # Formatted display
stmt.to_dataframe()                         # As DataFrame
stmt.canonical_type                         # Statement type

# ── Quick financial metrics ──
fin = Company("MSFT").get_financials()
fin.income_statement()                      # Income statement
fin.balance_sheet()                         # Balance sheet
fin.get_revenue()                           # Revenue value
```

## What's Next

You've learned how to parse XBRL financial data from SEC filings. Here are related tutorials:

- [Extract Financial Statements from SEC Filings](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/financial-statements-sec-python.ipynb)
- [Extract Revenue and Earnings from SEC Filings](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/extract-revenue-earnings-python.ipynb)
- [Compare Company Financials](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/compare-company-financials-python.ipynb)
- [Analyze 10-K Annual Reports](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/analyze-10k-annual-report-python.ipynb)

**Resources:**
- [EdgarTools Documentation](https://edgartools.readthedocs.io/)
- [GitHub Repository](https://github.com/dgunning/edgartools)
- [PyPI Package](https://pypi.org/project/edgartools/)

---

## Support EdgarTools

If you found this tutorial helpful, here are a few ways to support the project:

- **Star the repo** -- [github.com/dgunning/edgartools](https://github.com/dgunning/edgartools) -- it helps others discover edgartools
- **Visit edgartools.io** -- [edgartools.io](https://www.edgartools.io/) -- for more tutorials, articles, and updates
- **Report issues** -- found a bug or have a feature idea? [Open an issue](https://github.com/dgunning/edgartools/issues)
- **Share this notebook** -- know someone who works with SEC data? Send them the Colab link

*edgartools is free, open-source, and community-driven. No API key or paid subscription required.*